# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 8:30AM,240016,10,8170760,"Methapharm, Inc.",Released
1,Jun 21 2022 8:30AM,239998,10,0085783357,ISDIN Corporation,Released
2,Jun 21 2022 8:30AM,239998,10,0085783356,ISDIN Corporation,Released
3,Jun 21 2022 8:30AM,239998,10,0085783358,ISDIN Corporation,Released
4,Jun 21 2022 8:30AM,239998,10,0085783375,ISDIN Corporation,Released
5,Jun 21 2022 8:30AM,239998,10,0085783374,ISDIN Corporation,Released
6,Jun 21 2022 8:30AM,239998,10,0085783396,ISDIN Corporation,Released
7,Jun 21 2022 8:30AM,239998,10,0085783475,ISDIN Corporation,Released
8,Jun 21 2022 8:30AM,239998,10,0085783486,ISDIN Corporation,Released
9,Jun 21 2022 8:30AM,239998,10,0085783510,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,240007,Released,1
50,240008,Released,41
51,240009,Released,23
52,240014,Released,1
53,240016,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240007,NaN,NaN,1.0
240008,NaN,NaN,41.0
240009,NaN,NaN,23.0
240014,NaN,NaN,1.0
240016,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239892,0.0,0.0,1.0
239896,0.0,0.0,1.0
239897,0.0,0.0,1.0
239899,1.0,5.0,9.0
239901,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239892,0,0,1
239896,0,0,1
239897,0,0,1
239899,1,5,9
239901,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239892,0,0,1
1,239896,0,0,1
2,239897,0,0,1
3,239899,1,5,9
4,239901,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239892,,,1
1,239896,,,1
2,239897,,,1
3,239899,1,5,9
4,239901,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc."
1,Jun 21 2022 8:30AM,239998,10,ISDIN Corporation
33,Jun 21 2022 8:30AM,240008,10,ISDIN Corporation
75,Jun 21 2022 8:30AM,240009,10,ISDIN Corporation
76,Jun 21 2022 8:30AM,240006,10,ISDIN Corporation
112,Jun 20 2022 5:11PM,240014,21,"NBTY Global, Inc."
113,Jun 20 2022 4:20PM,240007,20,"ACI Healthcare USA, Inc."
114,Jun 20 2022 3:51PM,240004,19,ACG North America LLC
116,Jun 20 2022 3:49PM,240003,22,FUJIFILM Diosynth Biotechnologies Texas LLC
117,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc.",,,1
1,Jun 21 2022 8:30AM,239998,10,ISDIN Corporation,,,46
2,Jun 21 2022 8:30AM,240008,10,ISDIN Corporation,,,41
3,Jun 21 2022 8:30AM,240009,10,ISDIN Corporation,,,23
4,Jun 21 2022 8:30AM,240006,10,ISDIN Corporation,,,1
5,Jun 20 2022 5:11PM,240014,21,"NBTY Global, Inc.",,,1
6,Jun 20 2022 4:20PM,240007,20,"ACI Healthcare USA, Inc.",,,1
7,Jun 20 2022 3:51PM,240004,19,ACG North America LLC,,,2
8,Jun 20 2022 3:49PM,240003,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1
9,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd",,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc.",1,,
1,Jun 21 2022 8:30AM,239998,10,ISDIN Corporation,46,,
2,Jun 21 2022 8:30AM,240008,10,ISDIN Corporation,41,,
3,Jun 21 2022 8:30AM,240009,10,ISDIN Corporation,23,,
4,Jun 21 2022 8:30AM,240006,10,ISDIN Corporation,1,,
5,Jun 20 2022 5:11PM,240014,21,"NBTY Global, Inc.",1,,
6,Jun 20 2022 4:20PM,240007,20,"ACI Healthcare USA, Inc.",1,,
7,Jun 20 2022 3:51PM,240004,19,ACG North America LLC,2,,
8,Jun 20 2022 3:49PM,240003,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
9,Jun 20 2022 3:39PM,240002,19,"AdvaGen Pharma, Ltd",14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc.",1,,
1,Jun 21 2022 8:30AM,239998,10,ISDIN Corporation,46,,
2,Jun 21 2022 8:30AM,240008,10,ISDIN Corporation,41,,
3,Jun 21 2022 8:30AM,240009,10,ISDIN Corporation,23,,
4,Jun 21 2022 8:30AM,240006,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc.",1,NaN,NaN
1,Jun 21 2022 8:30AM,239998,10,ISDIN Corporation,46,NaN,NaN
2,Jun 21 2022 8:30AM,240008,10,ISDIN Corporation,41,NaN,NaN
3,Jun 21 2022 8:30AM,240009,10,ISDIN Corporation,23,NaN,NaN
4,Jun 21 2022 8:30AM,240006,10,ISDIN Corporation,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 8:30AM,240016,10,"Methapharm, Inc.",1,0.0,0.0
1,Jun 21 2022 8:30AM,239998,10,ISDIN Corporation,46,0.0,0.0
2,Jun 21 2022 8:30AM,240008,10,ISDIN Corporation,41,0.0,0.0
3,Jun 21 2022 8:30AM,240009,10,ISDIN Corporation,23,0.0,0.0
4,Jun 21 2022 8:30AM,240006,10,ISDIN Corporation,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4319488,208,1.0,75.0
12,239979,1,0.0,0.0
16,1439511,6,0.0,0.0
19,959943,18,0.0,0.0
20,1199775,46,41.0,5.0
21,719936,3,0.0,0.0
22,1199742,5,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4319488,208,1.0,75.0
1,12,239979,1,0.0,0.0
2,16,1439511,6,0.0,0.0
3,19,959943,18,0.0,0.0
4,20,1199775,46,41.0,5.0
5,21,719936,3,0.0,0.0
6,22,1199742,5,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,208,1.0,75.0
1,12,1,0.0,0.0
2,16,6,0.0,0.0
3,19,18,0.0,0.0
4,20,46,41.0,5.0
5,21,3,0.0,0.0
6,22,5,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,208.0
1,12,Released,1.0
2,16,Released,6.0
3,19,Released,18.0
4,20,Released,46.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,75.0,0.0,0.0,0.0,5.0,0.0,0.0
Executing,1.0,0.0,0.0,0.0,41.0,0.0,0.0
Released,208.0,1.0,6.0,18.0,46.0,3.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,75.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,41.0,0.0,0.0
2,Released,208.0,1.0,6.0,18.0,46.0,3.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,75.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,41.0,0.0,0.0
2,Released,208.0,1.0,6.0,18.0,46.0,3.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()